In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from pathlib import Path

In [2]:
db_path = Path("test.db")

if db_path.exists():
    db_path.unlink()

In [3]:
engine = create_engine("sqlite:///test.db")

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE IF NOT EXISTS test (a INT, b TEXT)"))

In [5]:
with engine.connect() as conn:
    conn.execute(text("INSERT INTO test (a, b) VALUES (1, 'foo')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (2, 'bar')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (3, 'baz')"))
    conn.commit()

In [6]:
df = pd.read_sql_query("SELECT * FROM test", engine)

df

,a,b
0,1,foo
1,2,bar
2,3,baz


In [7]:
# add new column to dataframe

df["c"] = [4, 5, 6]
df["d"] = ["qux", "quux", "quuz"]

df

,a,b,c,d
0,1,foo,4,qux
1,2,bar,5,quux
2,3,baz,6,quuz


In [8]:
from sqlmigrately import migrate_table

In [9]:
migrate_table("test", df, engine, push_data=True)

2024-11-23 19:21:25.779 | INFO     | sqlmigrately.main:migrate_table:21 - Detected new columns: [{'name': 'd', 'type': dtype('O')}, {'name': 'c', 'type': dtype('int64')}]
2024-11-23 19:21:25.780 | INFO     | sqlmigrately.utils:alter_table:85 - Executing: ALTER TABLE test ADD COLUMN d object
2024-11-23 19:21:25.802 | INFO     | sqlmigrately.utils:alter_table:85 - Executing: ALTER TABLE test ADD COLUMN c int64
2024-11-23 19:21:25.816 | INFO     | sqlmigrately.main:migrate_table:31 - Appending data to table: test


In [10]:
new_table = pd.read_sql_query("SELECT * FROM test", engine)

new_table

,a,b,d,c
0,1,foo,None,NaN
1,2,bar,None,NaN
2,3,baz,None,NaN
3,1,foo,qux,4.0
4,2,bar,quux,5.0
5,3,baz,quuz,6.0


In [11]:
# delete column from dataframe

del new_table["a"]

new_table

,b,d,c
0,foo,None,NaN
1,bar,None,NaN
2,baz,None,NaN
3,foo,qux,4.0
4,bar,quux,5.0
5,baz,quuz,6.0


In [12]:
migrate_table("test", new_table, engine, push_data=True, remove_cols=True)

2024-11-23 19:21:25.880 | INFO     | sqlmigrately.main:migrate_table:26 - Detected removed columns: [{'name': 'a', 'type': INTEGER()}]
2024-11-23 19:21:25.882 | INFO     | sqlmigrately.utils:alter_table:85 - Executing: ALTER TABLE test DROP COLUMN a
2024-11-23 19:21:25.903 | INFO     | sqlmigrately.main:migrate_table:31 - Appending data to table: test


In [13]:
final_table = pd.read_sql_query("SELECT * FROM test", engine)

final_table

,b,d,c
0,foo,None,NaN
1,bar,None,NaN
2,baz,None,NaN
3,foo,qux,4.0
4,bar,quux,5.0
5,baz,quuz,6.0
6,foo,None,NaN
7,bar,None,NaN
8,baz,None,NaN
9,foo,qux,4.0


In [ ]:
# test exceptions
migrate_table("test2", new_table, engine)

TableDoesNotExistError: Table test2 does not exist

In [14]:
# delete database
db_path.unlink()